----

## Материалы, использовавшиеся для выполнения лабораторной работы (программы, исходные файлы ?) 

1. [Нормализация данных](https://habr.com/ru/articles/527334/)
2. [Официальная документация Lasso-регрессии](https://scikit-learn.org/1.6/modules/generated/sklearn.linear_model.Lasso.html)
3. [Способы регцляризации](https://ranalytics.github.io/data-mining/042-Regularization.html)
4. [Официальная документация по применению PCA](https://scikit-learn.org/1.6/modules/generated/sklearn.decomposition.PCA.html)
5. [Официальная документация RDKit](https://www.rdkit.org/docs/source/rdkit.Chem.html)
6. Статьи с протоколами стандартизации химических данных:
    - https://pubs.acs.org/doi/10.1021/ci100176x
    - https://pubs.acs.org/doi/10.1021/acs.jcim.6b00129
7. [Официальная документация scikit-learn по стандартизации данных](https://scikit-learn.org/1.6/modules/generated/sklearn.preprocessing.StandardScaler.html)

----

# Поэтапное изложение процедуры


**Перед отбором дескрипторов:** 
1. Все молекулы были стандартизированы в соответсвии с базовым протоколом стандартизации. Стандартизация включала: 

* Удаление солей
* Нейтрализация зарядов
* Приведение к родительскому фрагменту
* Приведение к стандартной таутомерной форме
* Удаление стереохимии
* Сброс изотопов
* Канонизация SMILES
* Добавление и удаление гидрогенов
* Исправление металлов

_Стандартизация молекул_ — это процесс, направленный на приведение химических структур к единому формату или форме.

2. Были удалены выбросы в целевых значения. Так как модель не сможет хорошо обучиться на редких значениях. Редкие дискретные значения (для задач классификации) были удалены, если встречались во всем наборе данных менее 10%. А редкие непрерывные значения были удалены с использованием межквартильного размаха по стандартному способу. Для межквартильного размаха (IQR) были выбраны 75-й перцентиль (Q3) и 25-й перцентиль (Q1). Коэффициент порога выбросов был равен 1.5 * IQR. Так как целевые значения нельзя нормализовывать, то было принято решение удалить редкие значения.



**Перед обучением были сгенерированы различные виды дескрипторы:**

1.  Базовые (наиболее популярные) дескрипторы, которые включают в себя:

    _Структурные дескрипторы:_

* Количество тяжелых атомов
* Количество групп N-H или O-H
* Количество атомов азота и кислорода
* Число акцепторов водородных связей
* Число доноров водородных связей
* Количество гетероатомов
* Число вращаемых связей
* Число валентных электронов
* Число ароматических колец
* Число алифатических гетероциклов
* Общее количество колец

    _Физико-химические дескрипторы:_ 

* Полная молекулярная масса
* Логарифм P (гидрофобность)
* Молекулярная рефрактивность
* Топологическая полярная поверхность
* Индекс Балабана

Выбранные популярные дескрипторы помогают делать каждую молекулу более уникальной, тем самым решается проблема дубликатов.

2. Дескрипторы, отобранные с помощью L1-регуляризации (LASSO):

При обработке данных для построения модели использовался метод LASSO (Least Absolute Shrinkage and Selection Operator), основанный на L1-регуляризации. Этот подход позволяет:

* Выполнять отбор наиболее значимых дескрипторов за счет наложения штрафа на коэффициенты линейной регрессии.
* Исключать из набора признаков те дескрипторы, которые имеют наименьший вклад в предсказание целевой переменной (их коэффициенты стремятся к нулю).
* Сокращать размерность данных и предотвращать переобучение модели.

3. Дескрипторы, отобранные с помощью метода главных компонент (PCA):

Для уменьшения размерности данных и выделения наиболее информативных признаков использовался метод главных компонент (Principal Component Analysis, PCA). Этот метод основан на преобразовании исходных дескрипторов в новые переменные (главные компоненты), которые:

* Представляют линейные комбинации исходных дескрипторов.
* Максимизируют дисперсию данных, сохраняя как можно больше информации.
* Устраняют корреляцию между признаками, делая их ортогональными.

4. Отпечатки Моргана с радиусом = 3 и длиной битового вектора 256 и 512:

Для представления молекулярных структур использовались отпечатки Моргана (Morgan fingerprints) с параметрами:

* Радиус = 3: Определяет, насколько далеко от центрального атома молекулы происходит учет окружения для генерации отпечатка.
* Длина битового вектора = 256 и 512: Указывает размер итогового вектора, используемого для кодирования молекулярной структуры. Вектор длиной 256 битов позволяет компактно описать молекулу, тогда как вектор длиной 512 битов дает более детализированное представление.

5. Все возможные дескрипторы из rdkit 


**После отбора дескрипторов:**

1. Были удалены пропуски и дубликаты. 

* Удаление пропусков и дубликатов — это важный этап обработки данных. Пропуски (NaN) могут приводить к некорректным расчетам и снижению качества моделей, поэтому их удаление или корректная обработка позволяет улучшить результаты анализа. Дубликаты создают избыточность в данных, что может исказить выводы и увеличить время обучения модели, поэтому их исключение необходимо для оптимизации и повышения качества работы алгоритмов.

2. Для оценки влияния различных методов масштабирования на модель, дескрипторы были преобразованы двумя способами: с использованием MinMaxScaler, который нормирует значения в диапазон [0, 1], и StandardScaler, стандартизирующего данные относительно среднего и стандартного отклонения. Масштабирование нужно для устранения выбросов в дескрипторах.




-----

# Выводы

